In [42]:
%load_ext autoreload  
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
import pypianoroll
import matplotlib.pyplot as plt
import IPython
import pretty_midi
from midi2audio import FluidSynth
import os
import shutil
import glob
import numpy as np 
import pandas as pd
import pretty_midi
import pickle
import pypianoroll
from music21 import converter, instrument, note, chord, stream
import music21
# import librosa
import matplotlib.pyplot as plt
# from tensorflow.keras.utils import np_utils
import json
import IPython.display
from datetime import datetime
import mido
root_dir = '../raw_data'
from music_generator.data import get_npz_data, multitrack_to_midi, fetching_instrument_pianorolls
from music_generator.midi import sample_multitrack

In [43]:
data = get_npz_data(100, data_path='../raw_data/lpd_5/lpd_5_cleansed')

In [44]:
pianorolls = fetching_instrument_pianorolls(data, 'Piano')

In [45]:
midi_transfo = multitrack_to_midi(data, save_path='../raw_data/midi')

In [56]:
track = 
for standard in data[0]:
    if standard.name in 'Piano' and standard.pianoroll.shape[0] != 0:
        pianoroll_arrays.append(standard.pianoroll)

NameError: name 'multi' is not defined

In [50]:
test = sample_multitrack(data[0],data[0].tracks )

AttributeError: 'list' object has no attribute 'pianoroll'

In [7]:
X = fetching_instrument_pianorolls(data, 'Piano')

In [40]:
list_shape= []

for i in X:
    list_shape.append(len(i))

np.median(list_shape)

8856.0